# Humming search
Code from blog [Searching by Music: Leveraging Vector Search for Music Information Retrieval](https://www.elastic.co/fr/blog/searching-by-music-leveraging-vector-search-audio-information-retrieval)

**Author:** Alex Salgado

**Modified by:** David Pilato

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dadoonet/music-search/blob/main/elastic_music_search.ipynb)

## Settings

In [ ]:
import getpass

github_url = "https://github.com/dadoonet/music-search.git"
index_name = "my-audio-index"
audio_path = "/content/music-search/dataset"

es_cloud_id = getpass.getpass('Enter Elastic Cloud ID:  ')
es_user = "elastic"
# es_user = getpass.getpass('Enter cluster username:  ')
es_pass = getpass.getpass('Enter cluster password:  ')

## Setup

### Install files and dependencies

In [ ]:
!rm -rf music-search/
!git clone $github_url

In [ ]:
!pip install elasticsearch

In [ ]:
!pip install -qU panns-inference librosa

In [ ]:
# Import necessary modules for audio display from IPython
from IPython.display import Audio, display

# Import necessary modules for Audio Tagging (ML)
from panns_inference import AudioTagging

# Import Elasticsearch client
from elasticsearch import Elasticsearch

### Load the ML model

In [ ]:
# load the default ML model into the gpu/cpu.
model = AudioTagging(checkpoint_path=None, device='cuda') # change device to cpu if a gpu is not available

### Test connection with Elasticsearch

In [ ]:
# Connect to elasticsearch
es = Elasticsearch(cloud_id=es_cloud_id,
                   basic_auth=(es_user, es_pass)
                   )
es.info() # should return cluster info

### Create Elasticsearch index

In [ ]:
# Specify index configuration
index_config = {
  "mappings": {
    "_source": {
          "excludes": ["audio-embedding"]
      },
    "properties": {
      "audio-embedding": {
        "type": "dense_vector",
        "dims": 2048,
        "index": True,
        "similarity": "cosine"
      },
      "path": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256
          }
        }
      },
      "timestamp": {
        "type": "date"
      },
      "title": {
        "type": "text"
      },
      "genre": {
        "type": "text"
      }
    }
  }
}

# Delete existing index
if es.indices.exists(index=index_name):
    index_deletion = es.indices.delete(index=index_name)
    print("index deleted: ", index_deletion)

# Create index
index_creation = es.indices.create(index=index_name, body=index_config)
print("index created: ", index_creation)

## Code

### List audio files from music dir

In [ ]:
import os

def list_audio_files(directory):
    # The list to store the names of .wav files
    audio_files = []

    # Check if the path exists
    if os.path.exists(directory):
        # Walk the directory
        for root, dirs, files in os.walk(directory):
            for file in files:
                # Check if the file is a .wav file
                if file.endswith('.wav'):
                    # Extract the filename from the path
                    filename = os.path.splitext(file)[0]
                    print(filename)

                    # Add the file to the list
                    audio_files.append(file)
    else:
        print(f"The directory '{directory}' does not exist.")

    # Return the list of .wav files
    return audio_files

### Generate embeddings from a Wav file and normalize

In [ ]:
import numpy as np
import librosa

# Function to normalize a vector. Normalizing a vector means adjusting the values measured in different scales to a common scale.
def normalize(v):
   # np.linalg.norm computes the vector's norm (magnitude). The norm is the total length of all vectors in a space.
   norm = np.linalg.norm(v)
   if norm == 0:
        return v

   # Return the normalized vector.
   return v / norm

# Function to get an embedding of an audio file. An embedding is a reduced-dimensionality representation of the file.
def get_embedding (audio_file):

  # Load the audio file using librosa's load function, which returns an audio time series and its corresponding sample rate.
  a, _ = librosa.load(audio_file, sr=44100)

  # Reshape the audio time series to have an extra dimension, which is required by the model's inference function.
  query_audio = a[None, :]

  # Perform inference on the reshaped audio using the model. This returns an embedding of the audio.
  _, emb = model.inference(query_audio)

  # Normalize the embedding. This scales the embedding to have a length (magnitude) of 1, while maintaining its direction.
  normalized_v = normalize(emb[0])

  # Return the normalized embedding required for dot_product elastic similarity dense vector
  return normalized_v

### Index into Elasticsearch

In [ ]:
from datetime import datetime

# Storing Songs in Elasticsearch with Vector Embeddings:
def store_in_elasticsearch(song, embedding, path, index_name, genre, vec_field):
  body = {
      'audio-embedding' : embedding,
      'title': song,
      'timestamp': datetime.now(),
      'path' : path,
      'genre' : genre

  }

  es.index(index=index_name, document=body)
  print ("stored...",song, embedding, path, genre, index_name)

### Search for a similar song

In [ ]:
# Define a function to query audio vector in Elasticsearch
def query_audio_vector(es, emb, field_key, index_name):
    # Initialize the query structure
    # It's a bool filter query that checks if the field exists
    query = {
        "bool": {
            "filter": [{
                "exists": {
                    "field": field_key
                }
            }]
        }
    }

    # KNN search parameters
    # field is the name of the field to perform the search on
    # k is the number of nearest neighbors to find
    # num_candidates is the number of candidates to consider (more means slower but potentially more accurate results)
    # query_vector is the vector to find nearest neighbors for
    # boost is the multiplier for scores (higher means this match is considered more important)
    knn = {
        "field": field_key,
        "k": 5,
        "num_candidates": 100,
        "query_vector": emb,
        "boost": 100
    }

    # The fields to retrieve from the matching documents
    fields = ["title", "path", "genre", "body_content", "url"]

    # The name of the index to search
    index = index_name

    # Perform the search
    # index is the name of the index to search
    # query is the query to use to find matching documents
    # knn is the parameters for KNN search
    # fields is the fields to retrieve from the matching documents
    # size is the maximum number of matches to return
    # source is whether to include the source document in the results
    resp = es.search(index=index,
                     query=query,
                     knn=knn,
                     fields=fields,
                     size=5,
                     source=False)

    # Return the search results
    return resp


## Run

### List audio files

In [ ]:
audio_files = list_audio_files(audio_path)

### Read each file and index it

In [ ]:

# Initialize a list genre for test
genre_lst = ['jazz', 'opera', 'piano','prompt', 'humming', 'string', 'capella', 'eletronic', 'guitar']

for filename in audio_files:
  audio_file = audio_path + "/" + filename

  emb = get_embedding(audio_file)

  song = filename.lower()

  # Compare if genre list exists inside the song
  genre = next((g for g in genre_lst if g in song), "generic")

  store_in_elasticsearch(song, emb, audio_file, index_name, genre, 2 )


### Find a similar song (Bella Ciao)

In [ ]:
# Provide the URL of the audio file
audio_file = "/content/music-search/dataset/bella_ciao_humming.wav"

# Display the audio file in the notebook
Audio(audio_file)

In [ ]:
# Generate the embedding vector from the provided audio file
# 'get_embedding' is a function that presumably converts the audio file into a numerical vector
emb = get_embedding(audio_file)

In [ ]:
# Display the begining of the embeddings generated from the audio file
emb.tolist()

In [ ]:
# Query the Elasticsearch instance 'es' with the embedding vector 'emb', field key 'audio-embedding',
# and index name 'my-audio-index'
# 'query_audio_vector' is a function that performs a search in Elasticsearch using a vector embedding.
# 'tolist()' method is used to convert numpy array to python list if 'emb' is a numpy array.
resp = query_audio_vector (es, emb.tolist(), "audio-embedding", index_name)

In [ ]:
resp['hits']

In [ ]:
NUM_MUSIC = 5  # example value

for i in range(NUM_MUSIC):
    path = resp['hits']['hits'][i]['fields']['path'][0]
    genre = resp['hits']['hits'][i]['fields']['genre'][0]
    score = resp['hits']['hits'][i]['_score']
    score_padding = ('{: <10}'.format(score))
    genre_padding = ('{: <10}'.format(genre))
    print(f'{score_padding}', f'{genre_padding}', path)


In [ ]:
# Listen to the 2nd one (ignore the first as it's the same song)
print(resp['hits']['hits'][1]['fields']['path'][0])
Audio(resp['hits']['hits'][1]['fields']['path'][0])

In [ ]:
print(resp['hits']['hits'][2]['fields']['path'][0])
Audio(resp['hits']['hits'][2]['fields']['path'][0])

In [ ]:
print(resp['hits']['hits'][3]['fields']['path'][0])
Audio(resp['hits']['hits'][3]['fields']['path'][0])

In [ ]:
print(resp['hits']['hits'][4]['fields']['path'][0])
Audio(resp['hits']['hits'][4]['fields']['path'][0])

### Find a similar song (Mozart)

In [ ]:
# Provide the URL of the audio file
audio_file = "/content/music-search/dataset/mozart_symphony25_string-quartet.wav"

# Display the audio file in the notebook
Audio(audio_file)

In [ ]:
# Generate the embedding vector from the provided audio file
# 'get_embedding' is a function that presumably converts the audio file into a numerical vector
emb = get_embedding(audio_file)

In [ ]:
# Query the Elasticsearch instance 'es' with the embedding vector 'emb', field key 'audio-embedding',
# and index name 'my-audio-index'
# 'query_audio_vector' is a function that performs a search in Elasticsearch using a vector embedding.
# 'tolist()' method is used to convert numpy array to python list if 'emb' is a numpy array.
resp = query_audio_vector (es, emb.tolist(), "audio-embedding", index_name)

In [ ]:
for i in range(NUM_MUSIC):
    path = resp['hits']['hits'][i]['fields']['path'][0]
    genre = resp['hits']['hits'][i]['fields']['genre'][0]
    score = resp['hits']['hits'][i]['_score']
    score_padding = ('{: <10}'.format(score))
    genre_padding = ('{: <10}'.format(genre))
    print(f'{score_padding}', f'{genre_padding}', path)


In [ ]:
# Listen to the 2nd one (ignore the first as it's the same song)
print(resp['hits']['hits'][1]['fields']['path'][0])
Audio(resp['hits']['hits'][1]['fields']['path'][0])

In [ ]:
print(resp['hits']['hits'][2]['fields']['path'][0])
Audio(resp['hits']['hits'][2]['fields']['path'][0])

In [ ]:
print(resp['hits']['hits'][3]['fields']['path'][0])
Audio(resp['hits']['hits'][3]['fields']['path'][0])

In [ ]:
print(resp['hits']['hits'][4]['fields']['path'][0])
Audio(resp['hits']['hits'][4]['fields']['path'][0])